In [2]:
# RNN TRAINS THROUGH BACKPROPAGATION AND IT HAPPENS FOR EACH TIME STAMP
# KNOWN AS BACKPROPAGATION THROUGH TIME
# VANISHING AND EXPLODING GRADIENT
# THE CHANGE IN WEIGHT WILL BECOME LESSER AFTER MULTIPLE TIME STAMP(MULTIPLYING WITH LEARNING RATE)
# TO STOP EXPLODING GRADIENT - TRUNCATED BPN THROUGH TIME (BTT) (DOES NOT GO TILL THE STARTING,STARTS FROM SOMEWHERE INBETWEEN)
#                            - CLIP GRADIENT AFTER A CERTAIN THRESHOLD
#                            - RMS PROP TO ADJUST LEARNING RATE
# FOR VANISHING - ReLu
#               - RMSprop
#               - LSTM , GRU
# LSTMS- LONG SHORT TIME MEMORY NETWORKS - LEARNS LONG TIME DEPEDENCIES
# LSTMS HAVE FOUR INTERACTING LAYERS
# CELL STATE - LINEAR INTERACTION
# 1) SIGMOID LAYER (FORGET GATE LAYER) - DECIDES WHAT INFORMATION TO THROW AWAY FROM THE CELL STATE
#               - INPUTS VALUE FROM PREVIOUS TIME STAMP (H(t-1)) AND INPUT (x(t)) AND OUTPUTS A NUMBER BETWEEN 0 AND 1
#                  FOR EACH NUMBER IN THE CELL STATE
#               - f(t)=sigmoid(Wf*[H(t-1),X(t)]+B)
# 2) DECIDE WHAT INFORMATION TO STORE
#                HAS 2 PARTS :- i) SIGMOID LAYER TO DECIDE WHAT ALL TO UPDATE
#                              ii) CREATES A VECTOR OF THE CANDIDATE VALUES THAT WILL BE ADDED TO THE STATE LATER ON
#                BOTH VALUES FROM PREVIOUS STATE AND INPUT IS PASSED THROUGH THESE TWO LAYERS
#                BOTH PASS THROUGH SIGMOID TO GIVE I(t) AND BOTH PASS THROUGH tanh TO GIVE `Ç(t)
#                I(t) AND C(t) ARE MULTIPLIED AND ADDED TO CELL STATE
#                I(t)=sigmoid(Wi*[H(t-1),X(t)]+Bi)       `Ç(t)=tanh(Wc*[H(t-1),X(t)]+Bc)
#
# 3) OLD STATE C(t-1) IS UPDATED TO C(t)
#    C(t) = f(t)*C(t-1) + I(t)*`Ç(t)
#    1st TERM IS USED FOR FORGETTING THE TERMS THAT ARE NOT REQUIRED
#    2nd TERM GIVES US THE SCALED VALUE AS TO HOW MUCH WE DECIDE TO UPDATE EACH VALUE
# 4) DECIDING WHAT TO OUTPUT BASED ON OUR CELL STATE 
#    O(t)=sigmoid(Wo*[H(t-1),X(t)]+Bo)
#    h(t)= O(t)*tanh(C(t))

In [1]:
import numpy as np
import pandas as pd

In [4]:
import tensorflow as tf
import keras

In [ ]:
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense,LSTM

In [7]:
data=[]
target=[]
temp=[]
for i in range(100):
    for j in range(8):
        temp.append(i+j)
    data.append(temp)
    temp=[]
    target.append(i+5)
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
data_scaled=scaler.fit_transform(data)

In [6]:
data_scaled=np.array(data_scaled)
data_scaled=data_scaled.reshape([100,8,1])
target=np.array(target)
target=target.reshape([100,1])
from sklearn.preprocessing import OneHotEncoder
encoder=OneHotEncoder(categories='auto')
target_encoded=encoder.fit_transform(target)

In [7]:
target_encoded=target_encoded.toarray().reshape([100,100])

In [21]:
np.shape(target_encoded)

(100, 100)

In [9]:
x_train,x_test,y_train,y_test=train_test_split(data_scaled,target_encoded,random_state=42,test_size=.28)

In [137]:
model=Sequential()

In [138]:
# ( output size)
#(batch_input_shape-input shape of data) format (no of input,length of input sequence,length of each vector)
# if not known , can be replaced with None
# return sequence - 'True' gives output at each node | 'False' does not give output at each node
model.add(LSTM(1,batch_input_shape=(None,8,1),return_sequences=True))
model.add(LSTM(1,return_sequences=True))
model.add(LSTM(1,return_sequences=False))

In [139]:
model.add(Dense(128,activation='relu'))
model.add(Dense(100,activation='tanh'))

In [140]:
model.add(Dense(100,activation='softmax'))

In [141]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [177]:
results=model.fit(x_train,y_train,epochs=10,verbose=True)

Epoch 1/10
71/71 [==============================] - 0s 3ms/step - loss: 0.3253 - acc: 0.9577
Epoch 2/10
71/71 [==============================] - 0s 3ms/step - loss: 0.3424 - acc: 0.9437
Epoch 3/10
71/71 [==============================] - 0s 3ms/step - loss: 0.3471 - acc: 0.9296
Epoch 4/10
71/71 [==============================] - 0s 3ms/step - loss: 0.3022 - acc: 0.9718
Epoch 5/10
71/71 [==============================] - 0s 3ms/step - loss: 0.3088 - acc: 0.9718
Epoch 6/10
71/71 [==============================] - 0s 3ms/step - loss: 0.2954 - acc: 0.9718
Epoch 7/10
71/71 [==============================] - 0s 3ms/step - loss: 0.2987 - acc: 0.9718
Epoch 8/10
71/71 [==============================] - 0s 3ms/step - loss: 0.3019 - acc: 0.9577
Epoch 9/10
71/71 [==============================] - 0s 2ms/step - loss: 0.2921 - acc: 0.9859
Epoch 10/10
71/71 [==============================] - 0s 2ms/step - loss: 0.3027 - acc: 0.9859


In [178]:
y_predicted=model.predict(x_test)

In [179]:
l=[]
for i in range(len(y_predicted)):
    for j in range(len(y_predicted[0])):
        if max(y_predicted[i])==y_predicted[i][j]:
            l.append(1)
        else :
            l.append(0)
l=np.array(l)

In [180]:
# THE LENGTH OF INPUT SHOULD BE FLEXIBLE
# i) USE PADDING ii) USE (NONE,NONE,1) IN PLACE OF batch_input_shape

In [181]:
l=l.reshape([29,100])

In [182]:
encoder.inverse_transform(l)

array([[ 87],
       [ 59],
       [ 76],
       [ 51],
       [ 48],
       [ 43],
       [ 28],
       [ 86],
       [ 16],
       [  6],
       [ 24],
       [ 34],
       [ 79],
       [ 39],
       [ 94],
       [ 10],
       [ 80],
       [ 83],
       [ 16],
       [ 37],
       [ 61],
       [ 94],
       [ 32],
       [ 48],
       [ 73],
       [ 21],
       [ 46],
       [100],
       [ 13]])

In [183]:
encoder.inverse_transform(y_test)

array([[ 88],
       [ 58],
       [ 75],
       [ 50],
       [ 49],
       [ 44],
       [ 27],
       [ 85],
       [ 15],
       [  5],
       [ 23],
       [ 35],
       [ 78],
       [ 38],
       [ 95],
       [  9],
       [ 81],
       [ 82],
       [ 17],
       [ 36],
       [ 60],
       [ 93],
       [ 31],
       [ 47],
       [ 74],
       [ 20],
       [ 45],
       [101],
       [ 14]])

In [184]:
model.evaluate(x_test,y_test)

29/29 [==============================] - 0s 552us/step


[16.11809539794922, 0.0]

In [10]:
target

[5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104]